<div align="center">
<p align="center" style="width: 100%;">
    <img src="https://raw.githubusercontent.com/vlm-run/.github/refs/heads/main/profile/assets/vlm-black.svg" alt="VLM Run Logo" width="80" style="margin-bottom: -5px; color: #2e3138; vertical-align: middle; padding-right: 5px;"><br>
</p>
<p align="center"><a href="https://docs.vlm.run"><b>Website</b></a> | <a href="https://docs.vlm.run/"><b>API Docs</b></a> | <a href="https://docs.vlm.run/blog"><b>Blog</b></a> | <a href="https://discord.gg/AMApC2UzVY"><b>Discord</b></a>
</p>
<p align="center">
<a href="https://discord.gg/AMApC2UzVY"><img alt="Discord" src="https://img.shields.io/badge/discord-chat-purple?color=%235765F2&label=discord&logo=discord"></a>
<a href="https://twitter.com/vlmrun"><img alt="Twitter Follow" src="https://img.shields.io/twitter/follow/vlmrun.svg?style=social&logo=twitter"></a>
</p>
</div>

Welcome to **[VLM Run Cookbooks](https://github.com/vlm-run/vlmrun-cookbook)**, a comprehensive collection of examples and notebooks demonstrating the power of structured visual understanding using the [VLM Run Platform](https://app.vlm.run).


## Image Captioning & Tagging with VLM Run SDK

This notebook demonstrates how to use the **official VLM Run Node.js SDK** (`vlmrun` npm package) to generate detailed captions and tags for images. Image captioning is perfect for:
- **Accessibility**: Creating alt-text descriptions for images
- **Content Management**: Automatically organizing and cataloging image libraries
- **SEO**: Generating image descriptions for better search engine optimization
- **Automated Analysis**: Building workflows that understand visual content

We'll use VLM Run's vision-language models through the native SDK to generate comprehensive, contextual captions and extract relevant tags from various images.


### Environment Setup

To get started, install the VLM Run SDK and sign up for an API key on the [VLM Run App](https://app.vlm.run).
- Store the VLM Run API key under the `VLMRUN_API_KEY` environment variable.


## Prerequisites

* Node.js 18+
* VLM Run API key (get one at [app.vlm.run](https://app.vlm.run))
* Deno or tslab kernel for running TypeScript in Jupyter


## Setup

First, let's install the required packages:


In [ ]:
// Install the VLM Run SDK
// npm install vlmrun openai zod

// If using Deno kernel, install dependencies via npm specifiers
// For tslab, run: npm install vlmrun openai zod in your project directory


In [1]:
// Import the VLM Run SDK and dependencies
import { VlmRun } from "vlmrun";
import { z } from "zod";
import { zodToJsonSchema } from "zod-to-json-schema";


In [2]:
// Get API key from environment variable
const VLMRUN_API_KEY = Deno.env.get("VLMRUN_API_KEY");

if (!VLMRUN_API_KEY) {
    throw new Error("Please set the VLMRUN_API_KEY environment variable");
}

console.log("✓ API Key loaded successfully");


✓ API Key loaded successfully


Let's initialize the VLM Run client using the SDK


In [3]:
// Initialize the VLM Run client using the SDK
const client = new VlmRun({
    apiKey: VLMRUN_API_KEY,
    baseURL: "https://agent.vlm.run/v1"  // Use the agent API endpoint
});

console.log("✓ VLM Run SDK client initialized!");


✓ VLM Run SDK client initialized!


## Example 1: Simple Image Captioning with Agent Completions

The VLM Run SDK provides an OpenAI-compatible `agent.completions` interface that makes it easy to use familiar patterns.


In [4]:
// Example image URL
const imageUrl = "https://storage.googleapis.com/vlm-data-public-prod/hub/examples/image.object-detection/crossroad.jpg";

console.log("📷 Image URL:", imageUrl);
console.log("\n🔍 Generating caption using VLM Run SDK...\n");

// Use the agent.completions interface (OpenAI-compatible)
const response = await client.agent.completions.create({
    model: "vlm-agent-1",
    messages: [
        {
            role: "user",
            content: [
                { type: "text", text: "Generate a detailed caption for this image" },
                { type: "image_url", image_url: { url: imageUrl } }
            ]
        }
    ]
});

const caption = response.choices[0].message.content;
console.log(`Caption: ${caption}`);


📷 Image URL: https://storage.googleapis.com/vlm-data-public-prod/hub/examples/image.object-detection/crossroad.jpg

🔍 Generating caption using VLM Run SDK...

Caption: This detailed street scene captures a bustling urban environment, characterized by numerous pedestrians and a moderate flow of vehicles amidst tall city buildings. In the foreground, a family of four is prominently crossing a white-striped crosswalk: a bearded man in a light blue shirt and khaki pants, wearing earbuds and holding a phone; a woman in a dark jacket pushing a stroller with a baby; and a young child walking beside them. To their left, another man with a backpack, dressed in a dark coat and blue jeans, is also crossing. Further back on the right, a man in dark attire stands near a yellow taxi. Many other pedestrians are visible on the sidewalks and further down the street, indicating high foot traffic.

Vehicles include a prominent yellow Toyota RAV4 taxi stopped in the crosswalk, and a dark red Hyundai Santa

## Helper Function for Captioning

Let's create a reusable helper function using the VLM Run SDK:


In [5]:
/**
 * Generate a caption for an image using VLM Run SDK.
 * 
 * @param imageUrl - URL of the image to caption
 * @param prompt - Custom prompt for captioning (optional)
 * @returns Generated caption string
 */
async function generateCaption(
    imageUrl: string,
    prompt: string = "Generate a detailed caption for this image"
): Promise<string> {
    const response = await client.agent.completions.create({
        model: "vlm-agent-1",
        messages: [
            {
                role: "user",
                content: [
                    { type: "text", text: prompt },
                    { type: "image_url", image_url: { url: imageUrl } }
                ]
            }
        ]
    });
    
    return response.choices[0].message.content || "";
}

console.log("✓ Helper function defined");


✓ Helper function defined


## Example 2: Structured Output with Zod Schema

For production applications, you'll want structured JSON output. Let's use Zod schemas with the VLM Run SDK:


In [6]:
// Define the schema using Zod
const ImageCaptionSchema = z.object({
    caption: z.string().describe("Detailed caption of the scene"),
    tags: z.array(z.string()).describe("Tags that describe the image"),
    primary_objects: z.array(z.string()).describe("Main objects visible in the image"),
    scene_type: z.string().describe("Type of scene (e.g., indoor, outdoor, street, nature)"),
    mood: z.string().describe("Overall mood or atmosphere of the image")
});

// Type inference from schema
type ImageCaption = z.infer<typeof ImageCaptionSchema>;

console.log("✓ ImageCaption schema defined");
console.log("Schema:", JSON.stringify(zodToJsonSchema(ImageCaptionSchema), null, 2));


✓ ImageCaption schema defined
Schema: {
  "type": "object",
  "properties": {
    "caption": {
      "type": "string",
      "description": "Detailed caption of the scene"
    },
    "tags": {
      "type": "array",
      "items": {
        "type": "string"
      },
      "description": "Tags that describe the image"
    },
    "primary_objects": {
      "type": "array",
      "items": {
        "type": "string"
      },
      "description": "Main objects visible in the image"
    },
    "scene_type": {
      "type": "string",
      "description": "Type of scene (e.g., indoor, outdoor, street, nature)"
    },
    "mood": {
      "type": "string",
      "description": "Overall mood or atmosphere of the image"
    }
  },
  "required": [
    "caption",
    "tags",
    "primary_objects",
    "scene_type",
    "mood"
  ],
  "additionalProperties": false,
  "$schema": "http://json-schema.org/draft-07/schema#"
}


In [7]:
/**
 * Generate a structured caption using VLM Run SDK with Zod schema validation.
 * 
 * @param imageUrl - URL of the image to analyze
 * @returns Validated ImageCaption object
 */
async function generateStructuredCaption(imageUrl: string): Promise<ImageCaption> {
    const response = await client.agent.completions.create({
        model: "vlm-agent-1",
        messages: [
            {
                role: "user",
                content: [
                    { 
                        type: "text", 
                        text: "Analyze this image and provide a detailed caption, relevant tags, primary objects, scene type, and mood." 
                    },
                    { 
                        type: "image_url", 
                        image_url: { url: imageUrl, detail: "auto" } 
                    }
                ]
            }
        ],
        // Use VLM Run's response_format with schema at top level
        response_format: { 
            type: "json_schema", 
            schema: zodToJsonSchema(ImageCaptionSchema)
        } as any
    });
    
    const rawContent = response.choices[0].message.content || "{}";
    console.log("Raw JSON Response:");
    console.log(rawContent);
    console.log();
    
    // Parse and validate the response with Zod
    const parsed = JSON.parse(rawContent);
    return ImageCaptionSchema.parse(parsed);
}

// Generate structured caption
console.log("🔍 Generating structured caption...\n");
const structuredResult = await generateStructuredCaption(imageUrl);

console.log("✓ Validated Object:");
console.log(structuredResult);
console.log();
console.log("📝 Formatted Output:");
console.log(`Caption: ${structuredResult.caption}`);
console.log(`Tags: ${structuredResult.tags.join(", ")}`);
console.log(`Primary Objects: ${structuredResult.primary_objects.join(", ")}`);
console.log(`Scene Type: ${structuredResult.scene_type}`);
console.log(`Mood: ${structuredResult.mood}`);


🔍 Generating structured caption...

Raw JSON Response:
{"caption":"Pedestrians, including a family with a stroller, cross a busy city street in front of a yellow taxi cab, with numerous cars and towering buildings in the background, suggesting a bustling urban environment like New York City.","mood":"bustling","primary_objects":["pedestrians","yellow taxi","cars","buildings","street","crosswalk","stroller"],"scene_type":"urban street scene","tags":["city street","urban","pedestrians","traffic","taxi","crosswalk","buildings","daytime","New York City","cars","stroller","people"]}

✓ Validated Object:
{
  caption: "Pedestrians, including a family with a stroller, cross a busy city street in front of a yellow taxi cab, with numerous cars and towering buildings in the background, suggesting a bustling urban environment like New York City.",
  tags: [
    "city street",   "urban",
    "pedestrians",   "traffic",
    "taxi",          "crosswalk",
    "buildings",     "daytime",
    "New York 

## Example 3: Batch Processing Multiple Images

Process multiple images efficiently using the VLM Run SDK:


In [8]:
// Sample images to process
const sampleImages = [
    { url: "https://storage.googleapis.com/vlm-data-public-prod/hub/examples/image.object-detection/crossroad.jpg", name: "Street Scene" },
    { url: "https://images.unsplash.com/photo-1506905925346-21bda4d32df4?w=400", name: "Mountain" },
    { url: "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400", name: "Cat" },
];

interface BatchCaptionResult {
    imageName: string;
    caption: string;
    tags: string;
    sceneType: string;
    error?: string;
}

async function processImagesBatch(images: { url: string; name: string }[]): Promise<BatchCaptionResult[]> {
    const results: BatchCaptionResult[] = [];
    
    for (const img of images) {
        try {
            console.log(`Processing: ${img.name}...`);
            const result = await generateStructuredCaption(img.url);
            results.push({
                imageName: img.name,
                caption: result.caption,
                tags: result.tags.join(", "),
                sceneType: result.scene_type
            });
        } catch (error) {
            console.log(`Error processing ${img.name}: ${error}`);
            results.push({
                imageName: img.name,
                caption: `Error: ${error}`,
                tags: "",
                sceneType: "",
                error: String(error)
            });
        }
    }
    
    return results;
}

// Process all images
const batchResults = await processImagesBatch(sampleImages);

// Display results
console.log("\n=== Batch Processing Results ===\n");
console.table(batchResults.map(r => ({
    Image: r.imageName,
    Caption: r.caption.substring(0, 50) + "...",
    Tags: r.tags,
    Scene: r.sceneType
})));


Processing: Street Scene...
Raw JSON Response:
{"caption":"A busy urban street scene captures diverse pedestrians utilizing a crosswalk, with a man in a light blue shirt and khaki pants walking alongside a woman and two young children, one in a stroller. Another man with a backpack crosses on the left. Several yellow taxi cabs are visible on the street amidst other vehicles, including a red SUV and a white van, all surrounded by towering city buildings featuring advertisements and architectural details. The overall impression is a bustling daytime atmosphere in a major metropolitan area.","mood":"Bustling and dynamic, reflecting a typical active urban environment.","primary_objects":["people","cars","buildings","street"],"scene_type":"City street","tags":["city street","urban","New York City","pedestrians","traffic","cars","taxi","crosswalk","buildings","people","family","baby stroller","daytime","downtown"]}

Processing: Mountain...
Raw JSON Response:
{"caption":"A breathtaking mounta

## Example 4: Custom Caption Styles

Customize caption output by adjusting the prompt:


// Different caption styles using the VLM Run SDK
const captionStyles: Record<string, string> = {
    "Short & concise": "Generate a brief, one-sentence caption for this image.",
    "Detailed & descriptive": "Generate a detailed, descriptive caption for this image with at least 50 words.",
    "Creative & poetic": "Generate a creative and poetic caption for this image.",
    "SEO-optimized": "Generate an SEO-optimized caption for this image that includes relevant keywords.",
    "Accessibility focused": "Generate an accessibility-focused alt text for this image that would help visually impaired users understand the content."
};

console.log("Different Caption Styles for the Same Image:\n");
console.log("=".repeat(80));

for (const [styleName, prompt] of Object.entries(captionStyles)) {
    const captionResult = await generateCaption(imageUrl, prompt);
    console.log(`\n${styleName.toUpperCase()}:`);
    console.log(captionResult);
    console.log("-".repeat(80));
}


## Example 5: Using VLM Run SDK Image Predictions API

The VLM Run SDK also provides a dedicated `image` API for predictions. Let's explore this approach:


In [9]:
// Initialize a separate client for the main API (not agent API)
const vlmClient = new VlmRun({
    apiKey: VLMRUN_API_KEY,
    baseURL: "https://api.vlm.run/v1"  
});

console.log("✓ VLM Run main API client initialized!");

// Using the image predictions API
// Note: This uses the standard prediction API, not the agent API
console.log("\n📷 Using VLM Run Image API for captioning...\n");

try {
    // Generate caption using the image.generate method
    const imagePrediction = await vlmClient.image.generate({
        images: [imageUrl],
        model: "vlm-1",
        domain: "image.caption",
        config: {
            jsonSchema: zodToJsonSchema(ImageCaptionSchema)
        }
    });
    
    console.log("Image Prediction Response:");
    console.log(JSON.stringify(imagePrediction, null, 2));
} catch (error) {
    console.log("Note: Image predictions API requires specific domain access.");
    console.log("For general captioning, use the agent.completions API as shown in previous examples.");
    console.log("Error:", error);
}


✓ VLM Run main API client initialized!

📷 Using VLM Run Image API for captioning...

Image Prediction Response:
{
  "usage": {
    "elements_processed": 1,
    "element_type": "image",
    "credits_used": 2,
    "steps": null,
    "message": null,
    "duration_seconds": 0
  },
  "id": "6a2adca0-1513-4481-9ba4-4db18319b7db",
  "created_at": "2025-12-18T13:48:18.089268",
  "completed_at": "2025-12-18T13:48:26.965723Z",
  "response": {
    "caption": "A bustling city street scene, likely in New York, captures pedestrians crossing a wide intersection. In the foreground, a family with a child in a stroller walks across a crosswalk, accompanied by other individuals. Yellow taxi cabs and numerous other vehicles are visible in various lanes of traffic, surrounded by towering urban buildings and clear skies in the distance.",
    "mood": "Busy",
    "primary_objects": [
      "pedestrians",
      "yellow taxi cabs",
      "cars",
      "stroller",
      "city buildings",
      "crosswalk"
    

## VLM Run SDK Features

The VLM Run SDK (`vlmrun` npm package) provides several advantages over using the OpenAI client directly:

### Key Features

| Feature | Description |
|---------|-------------|
| `agent.completions` | OpenAI-compatible chat completions for agent API |
| `image.generate()` | Direct image prediction API access |
| `document.generate()` | Document processing capabilities |
| `audio.generate()` | Audio transcription and analysis |
| `video.generate()` | Video understanding and analysis |
| `files.upload()` | File upload for processing |
| `hub.list()` | Browse available models and domains |

### SDK Initialization Options

```typescript
const client = new VlmRun({
    apiKey: "your-api-key",
    baseURL: "https://api.vlm.run/v1",  // or "https://agent.vlm.run/v1"
    timeout: 60000,  // Request timeout in ms
    maxRetries: 3    // Number of retries for failed requests
});
```


## Example 6: Exploring Available Models

Use the SDK to discover available models and domains:


In [10]:
// List available models using the SDK
console.log("📋 Listing available models...\n");

try {
    const models = await vlmClient.models.list();
    console.log("Available Models:");
    models.forEach(model => {
        console.log(`  - ${model.id}: ${model.name || 'N/A'}`);
    });
} catch (error) {
    console.log("Could not list models:", error);
}

// List available hub domains
console.log("\n📋 Listing hub domains for image captioning...\n");

try {
    const hubItems = await vlmClient.hub.list({ 
        type: "image"  // Filter by image type
    });
    console.log("Available Image Domains:");
    hubItems.slice(0, 10).forEach(item => {
        console.log(`  - ${item.domain}: ${item.description || 'N/A'}`);
    });
    if (hubItems.length > 10) {
        console.log(`  ... and ${hubItems.length - 10} more`);
    }
} catch (error) {
    console.log("Could not list hub domains:", error);
}


📋 Listing available models...

Available Models:
  - undefined: N/A
  - undefined: N/A
  - undefined: N/A
  - undefined: N/A
  - undefined: N/A
  - undefined: N/A
  - undefined: N/A
  - undefined: N/A
  - undefined: N/A
  - undefined: N/A
  - undefined: N/A
  - undefined: N/A
  - undefined: N/A
  - undefined: N/A
  - undefined: N/A
  - undefined: N/A
  - undefined: N/A
  - undefined: N/A
  - undefined: N/A
  - undefined: N/A
  - undefined: N/A
  - undefined: N/A

📋 Listing hub domains for image captioning...

Could not list hub domains: TypeError: vlmClient.hub.list is not a function
    at <anonymous>:15:40
    at Object.runMicrotasks (ext:core/01_core.js:730:26)
    at processTicksAndRejections (ext:deno_node/_next_tick.ts:59:10)
    at runNextTicks (ext:deno_node/_next_tick.ts:76:3)
    at eventLoopTick (ext:core/01_core.js:194:21)


## Best Practices & Tips

### Caption Quality
- Be specific in your prompts about the level of detail you need
- For accessibility, ask for alt-text specifically
- For SEO, ask for keyword-rich descriptions

### SDK Usage
- Use `agent.completions` for OpenAI-compatible chat interface with image support
- Use `image.generate()` for domain-specific image predictions
- Set appropriate timeouts for large image processing
- Use retry logic for production applications

### Performance Tips
- Batch similar captioning tasks for efficiency
- Consider caching captions to avoid reprocessing
- Use appropriate image sizes (doesn't need to be full resolution)

### TypeScript Benefits
- Use Zod schemas for type-safe structured outputs
- Leverage TypeScript's type inference for better IDE support
- The VLM Run SDK is fully typed for excellent developer experience


## Additional Resources

- [VLM Run Documentation](https://docs.vlm.run)
- [VLM Run Node.js SDK](https://github.com/vlm-run/vlmrun-node-sdk)
- [SDK API Reference](https://docs.vlm.run/sdks/node-sdk)
- [API Reference](https://docs.vlm.run/agents/capabilities/image/captioning)
- [More Examples](https://github.com/vlm-run/vlmrun-cookbook)
- [Discord Community](https://discord.gg/AMApC2UzVY)

### Installation

```bash
npm install vlmrun openai zod zod-to-json-schema
```

### Quick Start

```typescript
import { VlmRun } from "vlmrun";

const client = new VlmRun({
    apiKey: process.env.VLMRUN_API_KEY,
    baseURL: "https://agent.vlm.run/v1"
});

const response = await client.agent.completions.create({
    model: "vlm-agent-1",
    messages: [
        {
            role: "user",
            content: [
                { type: "text", text: "Describe this image" },
                { type: "image_url", image_url: { url: "https://example.com/image.jpg" } }
            ]
        }
    ]
});

console.log(response.choices[0].message.content);
```
